In [2]:
# 📓 Notebook: wrangling_final.ipynb
# 🎯 Objetivo: pipeline único de wrangling desde el dataset crudo (places_county_2024.csv)
# -------------------------------------------------------------------

import pandas as pd

# ==============================
# 1. Cargar dataset crudo
# ==============================
file_path = "../data/places_county_2024.csv"
df = pd.read_csv(file_path, low_memory=False)

print("Dimensiones iniciales:", df.shape)

# ==============================
# 2. Pivotear datos (Question_Text -> columnas)
# ==============================
df_pivot = df.pivot_table(
    index=[
        "StateAbbr", "StateDesc", "LocationName",
        "TotalPopulation", "TotalPop18plus", "LocationID"
    ],
    columns="Short_Question_Text",
    values="Data_Value"
).reset_index()

print("Después de pivotear:", df_pivot.shape)

# ==============================
# 3. Eliminar duplicados
# ==============================
dups = df_pivot.duplicated().sum()
print(f"Duplicados encontrados: {dups}")
if dups > 0:
    df_pivot = df_pivot.drop_duplicates()

# ==============================
# 4. Renombrar columnas a snake_case
# ==============================
def to_snake_case(name):
    return (
        str(name)
        .strip()
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
        .replace("/", "_")
    )

df_pivot.columns = [to_snake_case(col) for col in df_pivot.columns]

# ==============================
# 5. Convertir tipos numéricos
# ==============================
df_pivot["totalpopulation"] = pd.to_numeric(df_pivot["totalpopulation"], errors="coerce").astype("Int64")
df_pivot["totalpop18plus"] = pd.to_numeric(df_pivot["totalpop18plus"], errors="coerce").astype("Int64")

for col in df_pivot.columns:
    if col not in ["stateabbr", "statedesc", "locationname", "locationid"]:
        df_pivot[col] = pd.to_numeric(df_pivot[col], errors="coerce")

# ==============================
# 6. Guardar dataset pivotado base
# ==============================
df_pivot.to_csv("../data/places_clean_final.csv", index=False)
print("\n✅ Guardado places_clean_final.csv:", df_pivot.shape)

# ==============================
# 7. Imputar sociales (mediana estatal)
# ==============================
cols_social = [
    "food_insecurity", "food_stamps", "housing_insecurity",
    "lack_of_social_emotional_support", "social_isolation",
    "transportation_barriers", "utility_services_threat"
]

df_imputed = df_pivot.copy()
for col in cols_social:
    if col in df_imputed.columns:
        df_imputed[col] = df_imputed.groupby("stateabbr")[col].transform(
            lambda x: x.fillna(x.median())
        )

df_imputed.to_csv("../data/places_imputed.csv", index=False)
print("✅ Guardado places_imputed.csv:", df_imputed.shape)

# ==============================
# 8. Eliminar sociales
# ==============================
df_no_social = df_pivot.drop(columns=cols_social, errors="ignore")
df_no_social.to_csv("../data/places_no_social.csv", index=False)
print("✅ Guardado places_no_social.csv:", df_no_social.shape)

# ==============================
# 9. Eliminar condados con nulos médicos
# ==============================
cols_with_nulls = [
    "high_cholesterol",
    "cholesterol_screening",
    "high_blood_pressure",
    "high_blood_pressure_medication"
]

# No social
df_no_social_clean = df_no_social.dropna(subset=cols_with_nulls)
df_no_social_clean.to_csv("../data/places_no_social_clean.csv", index=False)
print("\n✅ Guardado places_no_social_clean.csv:", df_no_social_clean.shape)

# Imputed (solo con sociales)
df_imputed_clean = df_imputed.dropna(subset=cols_with_nulls)
df_imputed_clean.to_csv("../data/places_imputed_clean.csv", index=False)
print("✅ Guardado places_imputed_clean.csv:", df_imputed_clean.shape)

# ==============================
# 10. Validación parcial de nulos
# ==============================
datasets = {
    "places_clean_final.csv": df_pivot,
    "places_no_social_clean.csv": df_no_social_clean,
    "places_imputed_clean.csv": df_imputed_clean
}

for name, data in datasets.items():
    total_nulls = data.isna().sum().sum()
    print(f"\n--- {name} ---")
    print("Dimensiones:", data.shape)
    print("🔍 Total de nulos:", total_nulls)
    if total_nulls == 0:
        print("✅ Sin nulos, dataset limpio")
    else:
        print("⚠️ Aún hay nulos")

# ==============================
# 11. Imputar todos los nulos (media nacional)
# ==============================
df_imputed_full = df_imputed.copy()

for col in df_imputed_full.columns:
    if col not in ["stateabbr", "statedesc", "locationname", "locationid"]:
        if df_imputed_full[col].isna().sum() > 0:
            df_imputed_full[col] = df_imputed_full[col].fillna(df_imputed_full[col].mean())

output_path = "../data/places_imputed_full_clean.csv"
df_imputed_full.to_csv(output_path, index=False)

print("\n✅ Guardado places_imputed_full_clean.csv:", df_imputed_full.shape)
print("🔍 Nulos restantes:", df_imputed_full.isna().sum().sum())

# ==============================
# 12. Resumen de todos los datasets generados
# ==============================
summary = []

datasets_paths = {
    "places_clean_final.csv": "../data/places_clean_final.csv",
    "places_imputed.csv": "../data/places_imputed.csv",
    "places_no_social.csv": "../data/places_no_social.csv",
    "places_no_social_clean.csv": "../data/places_no_social_clean.csv",
    "places_imputed_clean.csv": "../data/places_imputed_clean.csv",
    "places_imputed_full_clean.csv": "../data/places_imputed_full_clean.csv"
}

for name, path in datasets_paths.items():
    df_temp = pd.read_csv(path, low_memory=False)
    summary.append({
        "dataset": name,
        "rows": df_temp.shape[0],
        "columns": df_temp.shape[1],
        "nulls_total": df_temp.isna().sum().sum()
    })

df_summary = pd.DataFrame(summary)

# Guardar resumen en CSV
df_summary.to_csv("../data/wrangling_summary.csv", index=False)

print("\n📊 Resumen guardado en ../data/wrangling_summary.csv")
display(df_summary)


Dimensiones iniciales: (240886, 22)
Después de pivotear: (3144, 46)
Duplicados encontrados: 0

✅ Guardado places_clean_final.csv: (3144, 46)


c:\Users\erroc\OneDrive\Desktop\practicas\data science project\CityMind\env\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\erroc\OneDrive\Desktop\practicas\data science project\CityMind\env\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\erroc\OneDrive\Desktop\practicas\data science project\CityMind\env\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\erroc\OneDrive\Desktop\practicas\data science project\CityMind\env\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\erroc\OneDrive\Desktop\practicas\data science project\CityMind\env\Lib\site-packages\numpy\lib\nanfunctions.py:1215

✅ Guardado places_imputed.csv: (3144, 46)
✅ Guardado places_no_social.csv: (3144, 39)

✅ Guardado places_no_social_clean.csv: (3077, 39)
✅ Guardado places_imputed_clean.csv: (3077, 46)

--- places_clean_final.csv ---
Dimensiones: (3144, 46)
🔍 Total de nulos: 5357
⚠️ Aún hay nulos

--- places_no_social_clean.csv ---
Dimensiones: (3077, 39)
🔍 Total de nulos: 0
✅ Sin nulos, dataset limpio

--- places_imputed_clean.csv ---
Dimensiones: (3077, 46)
🔍 Total de nulos: 5089
⚠️ Aún hay nulos

✅ Guardado places_imputed_full_clean.csv: (3144, 46)
🔍 Nulos restantes: 0

📊 Resumen guardado en ../data/wrangling_summary.csv


,dataset,rows,columns,nulls_total
0,places_clean_final.csv,3144,46,5357
1,places_imputed.csv,3144,46,5357
2,places_no_social.csv,3144,39,268
3,places_no_social_clean.csv,3077,39,0
4,places_imputed_clean.csv,3077,46,5089
5,places_imputed_full_clean.csv,3144,46,0
